In [ ]:
# 1. IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve

In [5]:
# 2. LOAD DATA
data1 = pd.read_csv("data.csv")
data1.head(10)
data1.shape

(569, 33)

In [6]:
# 2. LOAD DATA
data2 = pd.read_csv("Breast_Cancer.csv")
data2.head(10)
data2.shape

(2509, 34)

In [ ]:
# 3. BASIC CLEANING
# - drop duplicates
# - fix column names
# - encode targets
# - check missing values
# - describe()

In [ ]:
# 4. EXPLORATORY ANALYSIS
# - class distribution
# - histograms
# - correlation heatmap
# - pairplots
# - boxplots

In [ ]:
# 5. DATA PREPROCESSING
# - scaling (if needed)
# - train/test split
# - categorical encoding
# - imputation if needed

In [ ]:
# 6. MODELING DATASET A
# - Logistic Regression
# - RandomForest
# - SVM
# - PCA + model
# - GridSearchCV
# - ROC curve and confusion matrix

In [ ]:
# 7. MODELING DATASET B
# - Logistic Regression baseline
# - RandomForest
# - GradientBoosting / XGBoost
# - Neural Network (MLP)
# - Hyperparameter tuning
# - ROC curve, confusion matrix

In [ ]:
# 8. FEATURE IMPORTANCE ANALYSIS
# - RF feature importance
# - SHAP (optional)

In [ ]:
# 9. RESULTS COMPARISON
# - summary tables
# - accuracy, F1, ROC-AUC vs model
# - visualisation

In [ ]:
# 10. CONCLUSIONS FOR REPORT
# - strengths and weaknesses
# - comparison of datasets
# - future work recommendations